In [75]:
import pandas as pd
import re
import json
import math
from datetime import datetime
from unidecode import unidecode 
import geopandas as gpd
from shapely.geometry import Point

In [76]:
foody_file = "D:/TRUNG/Raw/Foody/Hồ Chí Minh Foody/Foody_HoChiMinh_ThuDuc.csv"                    #Đọc dữ liệu crawl về từ Foody
ward_file = 'VietNam_Xa_fixed.shp'                                                              #Đọc dữ liệu shp để lấy phường, xã để bỏ vào Address bị thiếu phường, xã    
add_Ward_csv_file = 'D:/TRUNG/Raw/Foody/Hồ Chí Minh Foody/Foody_HoChiMinh_ThuDuc.csv'             #Xuất ra dữ liệu đã được thêm phường, xã
mongo_input_file = 'D:/TRUNG/Raw/MongoDB/Hồ Chí Minh/Mongo_HoChiMinh_ThuDuc_2024.json'                                #Đọc dữ liệu JSON từ MongoDB (chưa xử dụng được)
mongo_output_file= "Mongo_HoChiMinh_ThuDuc_2024_data.json"                                            #Xuất ra dữ liệu dạng JSON đã được xử lý  
compare_file = 'SoSanh_HoChiMinh_ThuDuc_2024(copy).csv'                                                     #Xuất ra dữ liệu trùng lặp từ Foody và MongoDB => Replace hoặc Duplicate
foody_addPoint_file= "D:/TRUNG/Clean/Add/Hồ Chí Minh/Foody_HoChiMinh_ThuDuc_2024_AddPoint.csv"            #Lọc trùng lặp và cho ra dữ liệu CẦN THÊM final (CẦN THÊM = Foody - Replace - Duplicate)

In [77]:
df = pd.read_csv(foody_file)
gdf = gpd.read_file(ward_file)

In [78]:
df['geometry'] = df.apply(lambda row: Point(row['Longitude'], row['Latitude']), axis=1)
points_gdf = gpd.GeoDataFrame(df, geometry='geometry', crs=gdf.crs)
result = gpd.sjoin(points_gdf, gdf, how='left', predicate='intersects')
result.rename(columns={'ten': 'Ward'}, inplace=True)
df['Ward'] = result['Ward']

In [79]:
def format_ward(ward):
    if pd.isna(ward):
        return None
    ward = re.sub(r'(?i)\bphường\b', 'P.', ward, flags=re.IGNORECASE)
    ward = re.sub(r'(?i)\bxã\b', 'Xã', ward, flags=re.IGNORECASE)
    ward = re.sub(r'(?i)\bthị trấn\b', 'TT.', ward, flags=re.IGNORECASE)
    return ward.strip()

df['Ward'] = df['Ward'].apply(format_ward)
def format_address(row):
    address = row['Address']
    ward = row['Ward']
    if re.search(r'\b(P\.|Xã|TT\.)', address, flags=re.IGNORECASE):
        return address
    if ward:
        return f"{address}, {ward}"
    return address

df['Address'] = df.apply(format_address, axis=1)
# display(df)
df.drop(columns=['geometry', 'Ward'], inplace=True)
df.to_csv(add_Ward_csv_file, index=False)


In [80]:
def extract_components(address):
    # Chuyển về dạng không dấu
    address = unidecode(address)

    # Tìm house number (nếu có)
    housenumber_match = re.match(r"^[\w/]+", address)
    housenumber = housenumber_match.group(0).strip().lower() if housenumber_match else ""

    # Tìm street và admin_level_4
    admin_match = re.search(r"\b(P\.|Xa|TT\.)\s?([\w\s]+)", address, flags=re.IGNORECASE)
    if admin_match:
        admin_level_4 = admin_match.group(2).strip().lower()  # Lấy tên địa phương, không kèm tiền tố
    else:
        admin_level_4 = ""

    # Loại bỏ admin_level_4 khỏi street (nếu có)
    if admin_level_4:
        address_without_admin = address.replace(admin_match.group(0), '').strip().lower()
        street = address_without_admin[len(housenumber):].strip()
    else:
        street = address[len(housenumber):].strip().lower()

    # Xóa dấu câu
    housenumber = re.sub(r"[^\w\s/]", "", housenumber)
    street = re.sub(r"[^\w\s]", "", street)
    admin_level_4 = re.sub(r"[^\w\s]", "", admin_level_4)

    return housenumber, street, admin_level_4

# Áp dụng hàm để tách các cột
df[["housenumber", "street", "admin_level_4"]] = df["Address"].apply(
    lambda x: pd.Series(extract_components(x))
)
display(df)

,Name,Address,District,City,Latitude,Longitude,Working hours,housenumber,street,admin_level_4
0,Gà rán và Mì Ý - Jollibee - Đặng Văn Bi,"102 Đặng Văn Bi, P. Bình Thọ",Tp. Thủ Đức,TP. HCM,10.844272,106.762181,09:00 - 22:00,102,dang van bi,binh tho
1,Mì Quảng Mỹ Sơn - Song Hành,"68 Song Hành, P. An Phú",Tp. Thủ Đức,TP. HCM,10.800610,106.740464,07:00 - 21:30,68,song hanh,an phu
2,"Highlands Coffee - Trà, Cà Phê & Bánh - Lê Văn...","331 Lê Văn Việt, P. Hiệp Phú",Tp. Thủ Đức,TP. HCM,10.845257,106.786560,07:00 - 20:00,331,le van viet,hiep phu
3,Sukiya - Cơm Bò Hầm & Mì Ramen - Vincom Lê Văn...,"Vincom Lê Văn Việt, 50 Lê Văn Việt, P. Hiệp Phú",Tp. Thủ Đức,TP. HCM,10.845345,106.778740,09:00 - 22:00,vincom,le van viet 50 le van viet,hiep phu
4,Trà Sữa Apa - Phú Châu,"54 Phú Châu, P. Tam Phú",Tp. Thủ Đức,TP. HCM,10.865171,106.746877,08:00 - 22:00,54,phu chau,tam phu
...,...,...,...,...,...,...,...,...,...,...
1918,Bún Bò O Sớm - Dân Chủ,"11 Dân Chủ, P. Bình Thọ",Tp. Thủ Đức,TP. HCM,10.845482,106.764991,06:00 - 18:00,11,dan chu,binh tho
1919,Bánh Canh Chả Cá Bình Định,"Đường Số 16, P. Linh Trung",Tp. Thủ Đức,TP. HCM,10.861073,106.781782,NaN,duong,so 16,linh trung
1920,Kiims Tea Coffee,"1643 Nguyễn Duy Trinh, P. Trường Thạnh",Tp. Thủ Đức,TP. HCM,10.809926,106.830026,08:00 - 21:00,1643,nguyen duy trinh,truong thanh
1921,Cơm Phong Sơn,"20/17 Đường 160, P. Tăng Nhơn Phú A",Tp. Thủ Đức,TP. HCM,10.838993,106.789708,09:00 - 20:00,20/17,duong 160,tang nhon phu a


In [81]:
with open(mongo_input_file , "r", encoding='utf-8') as file:
     raw_data = file.read()
processed_data1 = re.sub(r"NumberLong\((\d+)\)", r"\1", raw_data)
processed_data2 = re.sub(r"NumberInt\((\d+)\)", r"\1", processed_data1)
processed_data3 = re.sub(r'ISODate\("(.*?)"\)', r'"\1"', processed_data2)   
data = json.loads(processed_data3)
with open(mongo_output_file, "w", encoding="utf-8") as f:
    json.dump(data, f, ensure_ascii=False, indent=4)

In [82]:
def extract_admin_level_4(value):
    if value and isinstance(value, str):  # Kiểm tra giá trị hợp lệ
        match = re.fullmatch(r'phuong\s*(\d+)', value, re.IGNORECASE)  # Tìm 'phuong' và số
        if match:
            return match.group(1)  # Trả về phần số
    return value

In [83]:
extracted_data = [] 
for item in data:
    _id = item.get("_id")
    name = item.get("name")
    lng = item.get("location", {}).get("lng")
    lat = item.get("location", {}).get("lat")
    address_components = {comp["types"][0]: comp["searchName"] for comp in item.get("addressComponents", [])}
    housenumber = address_components.get("housenumber", None)
    street = address_components.get("street", None)
    admin_level_4 = extract_admin_level_4(address_components.get("admin_level_4", None))
    extracted_data.append({
        "_id": _id,
        "name": name,
        "lng": lng,
        "lat": lat,
        "housenumber_searchName": housenumber,
        "street_searchName": street,
        "admin_level_4_searchName": admin_level_4
    })
mongo_df = pd.DataFrame(extracted_data)

In [84]:
merged_df = pd.merge(
    mongo_df[['_id','name','housenumber_searchName', 'street_searchName', 'admin_level_4_searchName','lat','lng']], 
    df[['Name','housenumber', 'street', 'admin_level_4','Address',"District","City","Latitude","Longitude","Working hours"]], 
    left_on=['housenumber_searchName', 'street_searchName', 'admin_level_4_searchName'], 
    right_on=['housenumber', 'street', 'admin_level_4'], 
    how='inner'
)
merged_df = merged_df[merged_df['name'] != merged_df['Name']]
final_df = merged_df[["_id",'name','housenumber_searchName', 'street_searchName', 'admin_level_4_searchName','lat','lng','Name','Address',"District","City","Latitude","Longitude","Working hours"]]
# pd.set_option("display.max_rows", None)
display(final_df)

,_id,name,housenumber_searchName,street_searchName,admin_level_4_searchName,lat,lng,Name,Address,District,City,Latitude,Longitude,Working hours
0,65e048cd14770401f73d5900,Trạm Xe Bus Cầu Ông Dầu,416,quoc lo 13,hiep binh phuoc,10.836135,106.714012,Phở Huỳnh Mai,"416 Quốc Lộ 13, P. Hiệp Bình Phước",Tp. Thủ Đức,TP. HCM,10.836362,106.713831,09:00 - 23:00
1,65e048cd14770401f73d6608,Trạm Xe Bus Trung Tâm Dạy Nghề Quận 2,482,nguyen thi dinh,thanh my loi,10.779359,106.762927,Trung Bún - Bún Chả Hà Nội & Nem Cua Biển,"482 Nguyễn Thị Định, P. Thạnh Mỹ Lợi",Tp. Thủ Đức,TP. HCM,10.780286,106.762384,08:00 - 22:00
2,65e048cd14770401f73d660c,"Trạm Xe Bus 176, Lê Văn Thịnh",176,le van thinh,cat lai,10.778574,106.768747,Bánh Mì Lò Điện 176,"176 Lê Văn Thịnh, P. Cát Lái",Tp. Thủ Đức,TP. HCM,10.778752,106.768855,04:00 - 22:00
3,65e048cd14770401f73d681e,Trạm Xe Bus Công Ty Bình An,305,nguyen duy trinh,binh trung tay,10.787859,106.765501,Kichi Kichi Lẩu Băng Chuyền - Nguyễn Duy Trinh,"305 Nguyễn Duy Trinh, P. Bình Trưng Tây",Tp. Thủ Đức,TP. HCM,10.787677,106.765062,10:00 - 22:00
4,65e048cd14770401f73d6842,Trạm Xe Bus Công An Phường Linh Đông,525,kha van can,linh dong,10.845116,106.749763,Nem Thiên Hương Viên,"525 Kha Vạn Cân, P. Linh Đông",Tp. Thủ Đức,TP. HCM,10.845858,106.750455,07:00 - 22:00
5,65e048cd14770401f73d6845,Trạm Xe Bus Đại Học Văn Hoá Cơ Sở 2,287,do xuan hop,phuoc long b,10.823416,106.769954,Phở Vinh - Đỗ Xuân Hợp,"287 Đỗ Xuân Hợp, P. Phước Long B",Tp. Thủ Đức,TP. HCM,10.823270,106.770076,06:00 - 22:00
6,65e048cd14770401f73d68be,Trạm Xe Bus Trường Đại Học Sư Phạm Kỹ Thuật,1,vo van ngan,linh chieu,10.849871,106.772105,Cơm Chiên Dương Châu & Cơm Gà Xối Mỡ,"1 Võ Văn Ngân, P. Linh Chiểu",Tp. Thủ Đức,TP. HCM,10.851317,106.755623,16:00 - 22:30


In [85]:
def haversine(lat1, lng1, lat2, lng2):
    R = 6371.0  # Bán kính Trái đất (km)
    phi1 = math.radians(lat1)
    phi2 = math.radians(lat2)
    delta_phi = math.radians(lat2 - lat1)
    delta_lambda = math.radians(lng2 - lng1)
    a = math.sin(delta_phi / 2)**2 + math.cos(phi1) * math.cos(phi2) * math.sin(delta_lambda / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    distance_km = R * c
    distance_m = distance_km * 1000
    return round(distance_m, 2)
distances = []
final_df.loc[:, 'Distance'] = final_df.apply(lambda row: haversine(row['lat'], row['lng'], row['Latitude'], row['Longitude']), axis=1)
final_df.to_csv(compare_file, index= False)

C:\Users\Hoang Minh\AppData\Local\Temp\ipykernel_15216\440296866.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df.loc[:, 'Distance'] = final_df.apply(lambda row: haversine(row['lat'], row['lng'], row['Latitude'], row['Longitude']), axis=1)


In [86]:
df_filtered = df[~df['Name'].isin(final_df['Name'])]
df_filtered = df_filtered[["Name", "Address", "District", "City", "Latitude", "Longitude", "Working hours"]]
df_filtered.to_csv(foody_addPoint_file, index = False)
display(df_filtered)

,Name,Address,District,City,Latitude,Longitude,Working hours
0,Gà rán và Mì Ý - Jollibee - Đặng Văn Bi,"102 Đặng Văn Bi, P. Bình Thọ",Tp. Thủ Đức,TP. HCM,10.844272,106.762181,09:00 - 22:00
1,Mì Quảng Mỹ Sơn - Song Hành,"68 Song Hành, P. An Phú",Tp. Thủ Đức,TP. HCM,10.800610,106.740464,07:00 - 21:30
2,"Highlands Coffee - Trà, Cà Phê & Bánh - Lê Văn...","331 Lê Văn Việt, P. Hiệp Phú",Tp. Thủ Đức,TP. HCM,10.845257,106.786560,07:00 - 20:00
3,Sukiya - Cơm Bò Hầm & Mì Ramen - Vincom Lê Văn...,"Vincom Lê Văn Việt, 50 Lê Văn Việt, P. Hiệp Phú",Tp. Thủ Đức,TP. HCM,10.845345,106.778740,09:00 - 22:00
4,Trà Sữa Apa - Phú Châu,"54 Phú Châu, P. Tam Phú",Tp. Thủ Đức,TP. HCM,10.865171,106.746877,08:00 - 22:00
...,...,...,...,...,...,...,...
1918,Bún Bò O Sớm - Dân Chủ,"11 Dân Chủ, P. Bình Thọ",Tp. Thủ Đức,TP. HCM,10.845482,106.764991,06:00 - 18:00
1919,Bánh Canh Chả Cá Bình Định,"Đường Số 16, P. Linh Trung",Tp. Thủ Đức,TP. HCM,10.861073,106.781782,NaN
1920,Kiims Tea Coffee,"1643 Nguyễn Duy Trinh, P. Trường Thạnh",Tp. Thủ Đức,TP. HCM,10.809926,106.830026,08:00 - 21:00
1921,Cơm Phong Sơn,"20/17 Đường 160, P. Tăng Nhơn Phú A",Tp. Thủ Đức,TP. HCM,10.838993,106.789708,09:00 - 20:00
